In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
data = pd.read_excel("Bus-trips-data-sample.xlsx")

In [3]:
data.head()

Booking ID Journey Date Pickup Time Dropoff Time Booking Status  \
0     AD-674  01 Mar 2021       --:--     05:30 AM      Cancelled   
1     AD-840  01 Mar 2021       --:--     05:30 AM      Cancelled   
2    AD-1136  01 Mar 2021    04:28 AM     05:30 AM      Confirmed   
3    AD-1146  01 Mar 2021       --:--     05:30 AM      Cancelled   
4    AD-1175  01 Mar 2021       --:--     05:30 AM      Cancelled   

                      Pickup Address from Google Map  \
0              122A Sengkang E Way, Singapore 541122   
1              122A Sengkang E Way, Singapore 541122   
2    357A Admiralty Dr, Block 357A, Singapore 751357   
3  233 Jurong East Street 21, Block 233, Singapor...   
4  233 Jurong East Street 21, Block 233, Singapor...   

  Destination Address from Google Map  
0            Beach Station, Singapore  
1            Beach Station, Singapore  
2            Beach Station, Singapore  
3            Beach Station, Singapore  
4            Beach Station, Singapore

In [4]:
data.shape[0]

1277

In [5]:
# check data types
data.dtypes

Booking ID                             object
Journey Date                           object
Pickup Time                            object
Dropoff Time                           object
Booking Status                         object
Pickup Address from Google Map         object
Destination Address from Google Map    object
dtype: object

In [6]:
# Convert Journey Date, Pickup and Dropoff time to datetime format
data['Pickup Time'] = pd.to_datetime(data['Pickup Time'], format='%H:%M %p', errors='coerce')
data['Dropoff Time'] = pd.to_datetime(data['Dropoff Time'], format='%H:%M %p', errors='coerce')
data['Journey Date'] = pd.to_datetime(data['Journey Date'], format='%d %b %Y')

In [7]:
# Any null values 
data.isnull().sum()

Booking ID                               0
Journey Date                             0
Pickup Time                            527
Dropoff Time                           182
Booking Status                           0
Pickup Address from Google Map           0
Destination Address from Google Map      0
dtype: int64

**Drop rows with missing 'Pickup Time' and 'Dropoff Time'.**

In [8]:
data.dropna(inplace=True)

In [9]:
data.shape[0]

568

**568 rows of data for final processing.**

### Create Pickup Timeslot Feature
Each booking is assumed for up to 10 pax, thus requiring a 10 seater bus. For multiple bookings for the same trip within the same 15 minute brackets, bigger buses like 19 and 40 seaters can be despatched to fullfill these bookings.

In [10]:
def conv_time(ts):
    '''
    This function converts a string into timedelta format and returns it
    '''
    return datetime.strptime(ts, "%H:%M:%S")

In [11]:
# Define the 15 minute pickup time slots.
cut_labels = ['0:15AM','0:30AM','0:45AM','4:00AM', '4:15AM', '4:30AM', '4:45AM', '5:00AM', '5:15AM', '5:30AM', '5:45AM']
cut_bins = [conv_time("00:00:00"), conv_time("00:15:00"), conv_time("00:30:00"), conv_time("00:45:00"), 
            conv_time("04:00:00"), conv_time("04:15:00"), conv_time("04:30:00"), conv_time("04:45:00"),
            conv_time("05:00:00"), conv_time("05:15:00"), conv_time("05:30:00"), conv_time("05:45:00")]
data['Pickup Time Slot'] = pd.cut(data['Pickup Time'], bins=cut_bins, labels=cut_labels)

In [13]:
data[['Booking ID', 'Pickup Time Slot']].sort_values('Pickup Time Slot', ascending=True)

Booking ID Pickup Time Slot
942     AD-1730           0:15AM
1069    AD-1781           0:15AM
685     AD-1611           0:15AM
513     AD-1534           0:15AM
686     AD-1606           0:15AM
...         ...              ...
909     AD-1675              NaN
949     AD-1676              NaN
994     AD-1677              NaN
1141    AD-1881              NaN
1164    AD-1882              NaN

[568 rows x 2 columns]

**Bookings with the same journey and under the same 'Pickup Time Slot' can be combined so a bigger bus can be allocated.**

### Create Duration feature
Calculate the duration for each trip.

In [14]:
# create duration feature
data['Duration'] = (data['Dropoff Time'] - data['Pickup Time']).dt.seconds/60

In [15]:
data['Duration'] = data['Duration'].astype('Int32')

In [16]:
data.head()

Booking ID Journey Date         Pickup Time        Dropoff Time  \
2    AD-1136   2021-03-01 1900-01-01 04:28:00 1900-01-01 05:30:00   
6    AD-1255   2021-03-01 1900-01-01 04:48:00 1900-01-01 05:30:00   
7    AD-1279   2021-03-01 1900-01-01 04:56:00 1900-01-01 05:30:00   
8    AD-1284   2021-03-01 1900-01-01 04:55:00 1900-01-01 05:30:00   
9    AD-1285   2021-03-01 1900-01-01 05:17:00 1900-01-01 05:30:00   

  Booking Status                     Pickup Address from Google Map  \
2      Confirmed    357A Admiralty Dr, Block 357A, Singapore 751357   
6      Confirmed  666B Jurong West Street 65, Block 666B, Singap...   
7      Confirmed             683c Edgedale Plains, Singapore 821669   
8      Confirmed  986A Jurong West Street 93, Block 986A, Singap...   
9      Confirmed      93A Telok Blangah Street 31, Singapore 101093   

  Destination Address from Google Map Pickup Time Slot  Duration  
2            Beach Station, Singapore           4:30AM        62  
6            Beach Station, Singapore           5:00AM        42  
7            Beach Station, Singapore           5:00AM        34  
8            Beach Station, Singapore           5:00AM        35  
9            Beach Station, Singapore           5:30AM        13

In [17]:
data['Booking Status'].value_counts()

Confirmed    568
Name: Booking Status, dtype: int64

**Only 'Confirmed' bookings are included for processing.**

### Create Journey Feature
The 'Pickup Address' and 'Destination Address' are used to create the new feature 'Journey'.

In [18]:
# Create journey pairs
data['Pickup Destination Pairs'] = data['Pickup Address from Google Map'] + " TO " + data['Destination Address from Google Map']

In [19]:
# Do the same journeys have different durations
data.groupby(by=['Pickup Destination Pairs','Duration']).size()

Pickup Destination Pairs                                                              Duration
109 Whampoa Rd, Block 109, Singapore 321109 TO 1 HarbourFront Walk, Singapore 098585  20          2
111 Gangsa Rd, Block 111, Singapore 670111 TO 1 HarbourFront Walk, Singapore 098585   30          1
                                                                                      40          1
                                                                                      43          2
                                                                                      46          1
                                                                                                 ..
Beach Station, Singapore TO 93A Telok Blangah Street 31, Singapore 101093             20          1
Beach Station, Singapore TO 986A Jurong West Street 93, Block 986A, Singapore 641986  42          1
                                                                                      54          2
Singa

**Same journeys have different durations.**<br>
e.g. for the journey *'111 Gangsa Rd, Block 111, Singapore 670111 TO 1 HarbourFront Walk, Singapore 098585'*, there are durations under 45 minutes and durations over 45 minutes.<br><br>
As each trip journey and each bus total journey cannot exceed 45 minutes, we will split the data into journeys below 45 minutes and journeys above 45 minutes.

In [20]:
# Split bookings into those taking <=45 mins and those >45 mins
data_lt45 = data[data['Duration']<=45]
data_gt45 = data[data['Duration']>45]

In [21]:
print("Total bookings UNDER 45 minutes: ", data_lt45.shape[0])
print("Total bookings MORE THAN 45 minutes: ", data_gt45.shape[0])

Total bookings UNDER 45 minutes:  361
Total bookings MORE THAN 45 minutes:  207


In [22]:
# data_lt45.to_csv('journeys_lt45.csv', index=False)
# data_gt45.to_csv('journeys_gt45.csv', index=False)

### Get List of IDs

In [87]:
def get_booking_list(df, n):
    '''
    This function takes a dataframe and the number of buses required,
    n=1 for journeys below 45 minutes and n=2 for journeys above 45 minutes
    '''
    # Get list of unique days so that output is given by each day.
    days = df['Journey Date'].dt.strftime("%m-%d-%Y").unique()
    
    add_ids = []
    
    for day in days:
        
        # Get list of duration for the day
        durations = df[df['Journey Date'].dt.strftime("%m-%d-%Y")==day]['Duration'].unique()   

        for duration in durations:
            bus_10 = 0
            bus_19 = 0
            bus_40 = 0
            bus_10_ids=[]
            bus_19_ids=[]
            bus_40_ids=[]
            
            # Get list of unique journeys, for the day, with this duration
            pickup_dest_pairs = df[(df['Journey Date'].dt.strftime("%m-%d-%Y")==day) & 
                                   (df['Duration']==duration)]['Pickup Destination Pairs'].unique()
        
            for add in pickup_dest_pairs:
                        
                # Get list of pickup time slots, for the day, with this duration, for this journey
                pickup_time_slots = df[(df['Pickup Destination Pairs']==add) &
                                       (df['Journey Date'].dt.strftime("%m-%d-%Y")==day) & 
                                       (df['Duration']==duration)]['Pickup Time Slot'].unique()
                
                for timeslot in pickup_time_slots:
                    data_ids = df[(df['Pickup Destination Pairs']==add) &
                                  (df['Journey Date'].dt.strftime("%m-%d-%Y")==day) &
                                  (df['Duration']==duration) &
                                  (df['Pickup Time Slot']==timeslot)]
                    i = data_ids.shape[0]
                    if i > 0:
                        result = i // 4
                        start = 0
                        end = 1
                        while result > 0: # more than 4
                            bus_40 += n
                            bus_40_ids += [id for id in data_ids[start:end]['Booking ID']]
                            start += 4
                            end += 4
                            i -= 4
                            result = i // 4
                        result = i // 2
                        while result > 0: # more than 2
                            bus_19 += n
                            bus_19_ids += [id for id in data_ids[start:end]['Booking ID']]
                            start += 2
                            end += 2
                            i -= 2
                            result = i // 2
                        while i > 0: # more than 1
                            bus_10 += n
                            bus_10_ids += [id for id in data_ids[start:end]['Booking ID']]
                            start += 1
                            end += 1
                            i -= 1    
            add_id = {"date":day, "duration":duration, "bus_10":bus_10, "bus_10_ids":bus_10_ids, "bus_19":bus_19, "bus_19_ids":bus_19_ids,
                      "bus_40":bus_40, "bus_40_ids":bus_40_ids}
            add_ids += [add_id]
    return add_ids

In [88]:
data_lt45.head(5)

Booking ID Journey Date         Pickup Time        Dropoff Time  \
6     AD-1255   2021-03-01 1900-01-01 04:48:00 1900-01-01 05:30:00   
7     AD-1279   2021-03-01 1900-01-01 04:56:00 1900-01-01 05:30:00   
8     AD-1284   2021-03-01 1900-01-01 04:55:00 1900-01-01 05:30:00   
9     AD-1285   2021-03-01 1900-01-01 05:17:00 1900-01-01 05:30:00   
12     AD-828   2021-03-01 1900-01-01 00:30:00 1900-01-01 00:57:00   

   Booking Status                     Pickup Address from Google Map  \
6       Confirmed  666B Jurong West Street 65, Block 666B, Singap...   
7       Confirmed             683c Edgedale Plains, Singapore 821669   
8       Confirmed  986A Jurong West Street 93, Block 986A, Singap...   
9       Confirmed      93A Telok Blangah Street 31, Singapore 101093   
12      Confirmed                           Beach Station, Singapore   

                  Destination Address from Google Map Pickup Time Slot  \
6                            Beach Station, Singapore           5:00AM   
7                            Beach Station, Singapore           5:00AM   
8                            Beach Station, Singapore           5:00AM   
9                            Beach Station, Singapore           5:30AM   
12  769 Bedok Reservoir View, Block 769, Singapore...           0:30AM   

    Duration                           Pickup Destination Pairs  
6         42  666B Jurong West Street 65, Block 666B, Singap...  
7         34  683c Edgedale Plains, Singapore 821669 TO Beac...  
8         35  986A Jurong West Street 93, Block 986A, Singap...  
9         13  93A Telok Blangah Street 31, Singapore 101093 ...  
12        27  Beach Station, Singapore TO 769 Bedok Reservoi...

In [89]:
lt_45 = get_booking_list(data_lt45, 1)

In [90]:
df_lt45 = pd.DataFrame.from_dict(lt_45).set_index(['date', 'duration'])
df_lt45.shape[0]

303

In [91]:
gt_45 = get_booking_list(data_gt45, 2)

In [92]:
df_gt45 = pd.DataFrame.from_dict(gt_45).set_index(['date', 'duration'])
df_gt45.shape[0]

188

In [93]:
df_final = pd.concat([df_lt45, df_gt45])

In [94]:
df_final.sort_index(inplace=True)

In [95]:
df_final.head(10)

bus_10          bus_10_ids  bus_19 bus_19_ids  bus_40  \
date       duration                                                          
03-01-2021 13             1           [AD-1285]       0         []       0   
           17             1           [AD-1278]       0         []       0   
           19             1           [AD-1308]       0         []       0   
           27             1            [AD-828]       0         []       0   
           30             1           [AD-1310]       0         []       0   
           34             1           [AD-1279]       0         []       0   
           35             1           [AD-1284]       0         []       0   
           36             2  [AD-1271, AD-1281]       0         []       0   
           37             1           [AD-1108]       0         []       0   
           41             1           [AD-1296]       0         []       0   

                    bus_40_ids  
date       duration             
03-01-2021 13               []  
           17               []  
           19               []  
           27               []  
           30               []  
           34               []  
           35               []  
           36               []  
           37               []  
           41               []

In [96]:
def print_out(df):
    '''
    This function takes in the final dataframe and outputs in the format required
    '''
    buses = ["10", "19", "40"]
    journey_dates = []
    type_of_bus = []
    total_journey_time = []
    booking_ids = []
    
    for index, row in df.iterrows():
        for bus in range(len(buses)):
            if row["bus_" + buses[bus]] > 0:
                journey_dates.append(index[0])
                type_of_bus.append(buses[bus])
                total_journey_time.append(str(index[1]))
                booking_ids.append(row["bus_" + buses[bus] + "_ids"])
                
    return pd.DataFrame({"Journey_Date":journey_dates, "Type_of_Bus_10_19_40_seaters":type_of_bus, 
                         "Total_Journey_Time_In_Minutes":total_journey_time,
                         "Array_Of_Booking_IDs_To_Board_The_Bus":booking_ids})

In [97]:
df_output = print_out(df_final)

In [98]:
df_output['Journey_Date'] = pd.to_datetime(df_output['Journey_Date']).dt.strftime('%d %b %Y')

In [100]:
df_output.to_excel("output_plan.xlsx", index=False)